In [9]:
import os
import pandas as pd
import plotly.express as px
import progressbar
import json
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
pd.set_option('display.max_colwidth', None)

In [10]:
data_path = '../src/data'
mapping_corpus_path = data_path + r'/product/lspc2020_to_tablecorpus'
mapping_corpus_path_2 = data_path + r'/product/lspc2020_to_tablecorpus/Cleaned'
table_corpus_path = data_path + r'/product/product_top100/cleaned'
table_corpus_path_with_id = data_path + r'/product/product_top100/cleaned/with_id'
table_corpus_path2 = data_path + r'/product/product_minimum3/cleaned/with_id'

In [27]:
df_joined_clothes = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Clothes_clusters_all_8_tables_post_processed.csv'))
df_joined_clothes['category']='clothes'

In [28]:
df_joined_electronics = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Electronics_clusters_all_8_tables_post_processed.csv'))
df_joined_electronics['category']='electronics'

In [29]:
df_joined_bikes = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Bikes_clusters_all_8_tables_post_processed.csv'))
df_joined_bikes['category']='bikes'

In [30]:
df_joined_cars = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Cars_clusters_all_8_tables_post_processed.csv'))
df_joined_cars['category']='cars'

In [31]:
df_joined_technology = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Technology_clusters_all_8_tables_post_processed.csv'))
df_joined_technology['category']='technology'

In [32]:
df_joined_drugstore = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Drugstore_clusters_all_8_tables_post_processed.csv'))
df_joined_drugstore['category']='drugstore'

In [33]:
df_joined_tools = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Tools_clusters_all_8_tables_post_processed.csv'))
df_joined_tools['category']='tools'

In [34]:
df_joined_random = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Random_clusters_all_8_tables_post_processed.csv'))
df_joined_random['category']='random'

In [35]:
frames = [df_joined_electronics, df_joined_clothes,df_joined_bikes,df_joined_cars,df_joined_technology,df_joined_drugstore,df_joined_tools,df_joined_random]
df_concat = pd.concat(frames).drop(columns = ['Unnamed: 0','Unnamed: 0.1','Valid'])
df_concat

,cluster_id,url,row_id,table_id,name,description,brand_x,brand_y,tokens,category
0,289820,https://www.tinkco.com/fr/canon-pgi-555pgbk-xxl-cartouche-d-encre-canon-8049b001-noir-pigmentee-37ml-pgi555pgbkxxl,2502,Product_tinkco.com_September2020.json.gz,canon pgi-555pgbk xxl - cartouche d'encrecanon 8049b001 noir pigmentée 37ml,Cartouche originale Canon PGI-555PGBK XXL (PGI555PGBKXXL / 8049B001) de très grande capacité pour imprimante Canon MX725 et MX925. Couleur : noir p...,NaN,NaN,"['canon', 'pgi-555pgbk', 'xxl', 'cartouche', ""d'encrecanon"", '8049b001', 'noir', 'pigmentée', '37ml']",electronics
1,9885,https://www.tecnoprecios.net/t%C3%B3ner-original/497-toner-canon-723-amarillo-8500-paginas.html,607,Product_tecnoprecios.net_September2020.json.gz,toner canon 723 amarillo 8500 páginas,TONER CANON 723 AMARILLO 8500 PáGINAS LBP7750CDN,NaN,NaN,"['toner', 'canon', '723', 'amarillo', '8500', 'páginas']",electronics
2,323989,https://store.macs-4-u.com.au/usb-c-charge-cable-1-m.html,192,Product_macs-4-u.com.au_September2020.json.gz,usb-c charge cable (1 m),USB-C Charge Cable (1 m),NaN,NaN,"['usb-c', 'charge', 'cable', '1']",electronics
3,210849,https://www.cingolanibikeshop.com/specialized-sella-power-comp-nero-143.html,447,Product_cingolanibikeshop.com_September2020.json.gz,specialized sella power comp nero 143,SELLA POWER COMP NERO 143,Specialized,NaN,"['specialized', 'sella', 'power', 'comp', 'nero', '143']",electronics
4,345616,https://www.scan.co.uk/products/6tb-wd-red-pro-wd6003ffbx-35-nas-hdd-sata-iii-6gb-s-7200rpm-256mb-cache-shock-sensor-ncq-oem,4945,Product_scan.co.uk_September2020.json.gz,"wd red pro 6tb 3.5\"" sata nas hdd/hard drive","6TB WD Red Pro WD6003FFBX, 3.5\"" NAS HDD, SATA III 6Gb/s, 7200rpm, 256MB Cache, Shock Sensor, NCQ, OEM",WD,NaN,"['wd', 'red', 'pro', '6tb', '3.5\\', ""''"", 'sata', 'hdd/hard', 'drive']",electronics
...,...,...,...,...,...,...,...,...,...,...
21937,36306411,https://www.mainstreetfloorcoveringvermont.com/american-olean-mirasol-whites--creams-silver-marble-12-tile-flooring-ml7212121p,35,Product_mainstreetfloorcoveringvermont.com_September2020.json.gz,mirasol silver marble ml72,"Main Street Floor Covering in Essex Junction has a top selection of American Olean Ceramic & Porcelain Tile Flooring, including Mirasol™ Silver Marble in 12\"" x 12\""",American Olean,american olean,"['mirasol', 'silver', 'marble', 'ml72']",random
21938,62520180,https://www.thefloordrs.com/american-olean-quarry-naturals-beiges--browns-desert--2-tile-flooring-0n0322chipa,59,Product_thefloordrs.com_September2020.json.gz,quarry naturals desertn03 1,"The Floor Doctors in Des Moines has a top selection of American Olean Ceramic & Porcelain Tile Flooring, including Quarry Naturals® Desert * in 2\"" x 2\""",American Olean,american olean,"['quarry', 'naturals', 'desertn03', '1']",random
21939,56187443,https://www.amitybicycles.com/product/garmin-edge-510-silicone-case-198697-1.htm,353,Product_amitybicycles.com_September2020.json.gz,edge 510 silicone case,"Protect your Garmin Edge 510 with a form-fitting, silicone case and keep your mind at ease. Choose from a variety of cool colors to match your personal style.",NaN,NaN,"['edge', '510', 'silicone', 'case']",random
21940,2778331,https://www.curiousemporium.co.uk/products/lego-75957-harry-potter-the-knight-bus,59,Product_curiousemporium.co.uk_September2020.json.gz,lego 75957 harry potter the knight bus,"Give your young witch or wizard the ride of their life aboard the LEGO® Harry Potter™ 75957 Knight Bus™! When kids catch the purple triple-decker bus, they know they're in for a wild ride. With a hinged side panel for easy access to the interior, and a removable roof to reveal the upper deck, this magical toy bus playset is bursting with play potential. Once Harry's trunk has been loaded, it's time to hold tight. As the vehicle races away, swerving around corners and bouncing over bumps, kids will love how the bed slides back and forth and the chandelier swings from the ceiling.

In [76]:
df_final = pd.read_excel("Final_v2.xlsx")

In [77]:
df_clusters = df_concat[df_concat['cluster_id'].isin(df_final['cluster_id'].to_list())]

In [78]:
df_clusters

,cluster_id,url,row_id,table_id,name,description,brand_x,brand_y,tokens,category
3,16617,https://simplifinetworks.com/v2/point-to-multi-point/266-nanobeam-ac-gen2.html,167,Product_simplifinetworks.com_September2020.json.gz,nanobeam-ac-gen2,NBE-5AC-Gen2The sleek NanoBeam design with proprietary airMAX ac chipset and dedicated management Wi-Fi for easy U Mobile app support and fast setup. This airMAX ac CPE pivots on a ball joint for mounting and alignment flexibility.DATA SHEET,NaN,NaN,['nanobeam-ac-gen2'],electronics
4,323989,https://store.macs-4-u.com.au/usb-c-charge-cable-1-m.html,192,Product_macs-4-u.com.au_September2020.json.gz,usb-c charge cable (1 m),USB-C Charge Cable (1 m),NaN,NaN,"['usb-c', 'charge', 'cable', '1']",electronics
6,345616,https://www.scan.co.uk/products/6tb-wd-red-pro-wd6003ffbx-35-nas-hdd-sata-iii-6gb-s-7200rpm-256mb-cache-shock-sensor-ncq-oem,4945,Product_scan.co.uk_September2020.json.gz,"wd red pro 6tb 3.5\"" sata nas hdd/hard drive","6TB WD Red Pro WD6003FFBX, 3.5\"" NAS HDD, SATA III 6Gb/s, 7200rpm, 256MB Cache, Shock Sensor, NCQ, OEM",WD,NaN,"['wd', 'red', 'pro', '6tb', '3.5\\', ""''"", 'sata', 'hdd/hard', 'drive']",electronics
7,213496,https://de.camerok.com/Sony_SEL30M35_camera_lense,644,Product_camerok.com_September2020.json.gz,sony sel30m35 camera lense,"Dieses Objektiv wurde entwickelt, um bieten vielseitige Hochleistungs-makro-Funktionen in einem kompakten, leichten Körper. Es ist eine echte 1:1 makro-Objektiv mit einer erstaunlichen Naheinstellgrenze von nur knapp 1 Zoll.Die SEL30M35 wurde entwickelt, um bieten vielseitige Hochleistungs-makro-Funktionen in einem Objektiv ist kompakt, leicht und einfach zu bedienen. Es ist eine echte 1:1 makro-Objektiv mit 2,4 Zentimeter minimale Arbeitsabstand ermöglicht winzigen Themen und details zu erbringenden mit hervorragender Auflösung und Kontrast.1:1 AbbildungsmaßstabDie SEL30M35 hat eine 1:1-Abbildungsmaßstab so, dass die Probanden erfasst werden können, werden in voller Größe auf dem Bildsensor der Kamera. Dies macht es einfach, um zu Schießen Qualität makro-Nahaufnahmen von Blumen, Insekten und kleine Objekte.Asphärische/ED-Glas-ElementeDie SEL30M35 bietet eine hervorragende Bildqualität über den gesamten Bildbereich, mit hohem Kontrast und minimale chromatische aberration von unendlich bis 1:1 Vergrößerung bei allen Blendeneinstellungen. ED-Glas verwendet wird, für die G4-element, minimieren die chromatische aberration und die Farbe Blutungen sowie multi-Beschichtung während für besonders saubere, klare rendering.Gemacht für FilmeDie SEL30M35 ebenso führt auch bei Videoaufnahmen. Dank eines internen Schrittmotor und hinten-mit Schwerpunkt design ist optimiert für video-performance für Laufruhe Grundvoraussetzung für hochwertige Film zu erfassen. Darüber hinaus werden die filter Halterung nicht drehen, während der Schwerpunkt für die problemlose polarisierten filter verwenden.Glatte Manuelle FokussierungEin easy-grip-Fokus-ring auf dem Objektivtubus gibt Ihnen eine Feste \""hands-on\"" fühlen sich in der manuellen Fokussierung. Zusätzlich, Direct Manual Focus (DMF) können Sie den Fokus manuell nach AF lock-on, ohne die Notwendigkeit für einen Wechsel. Dies kann sehr nützlich in Situationen, in denen der AF nicht sperren auf das Thema genau, oder wenn AF Schlösser auf einen Punkt, der nicht mit Ihrer Absicht. Elektronische manueller Fokus-Technologie wurde ursprünglich entwickelt für den professionellen Sony Camcorder, um glatte, präzise Fokussierung.",Sony,NaN,"['sony', 'sel30m35', 'camera', 'lense']",electronics
8,167469,https://www.boulanger.com/ref/9000224731?xtor=AL-6875-%5B7%5D-%5B1395090794%5D-%5Bdeeplink%5D&utm_medium=affiliation&utm_source=effinity&utm_campaign=7&utm_content=1395090794,6052,Product_boulanger.com_September2020.json.gz,caméra de sécurité xiaomi mi home security camera 360degrés 1080,Xiaomi -Caméra de sécurité Domestique 360° 1080P- Blanc,NaN,NaN,"['caméra', 'sécurité', 'xiaomi', 'home', 'security', 'camera', '360degrés', '1

In [79]:
df_clusters[df_clusters['cluster_id']==5121322
]

,cluster_id,url,row_id,table_id,name,description,brand_x,brand_y,tokens,category
23,5121322,https://www.akcomicshop.com/7-wonders.html,862,Product_akcomicshop.com_September2020.json.gz,asmodee 7 wonders,Base Game3-7 playersAges 13+30 mins,Asmodee,NaN,"['asmodee', '7', 'wonders']",electronics
28,5121322,https://monsterblooms.com/product/tri-color-ti-hawaiian-ti-plant-cordyline,198,Product_monsterblooms.com_September2020.json.gz,tri color ti hawaiian ti plant (cordyline),Tricolor Foliage. Cordyline Tricolor is a mid sized easy to grow Cordyline. Leaves sport pink cream green and maroon striated coloration which darken slightly with age. Mature height is around 4 feet. Hawaiian Ti plants are the perfect plant choice to add lush tropical colors to any indoor or outdoor environment. Easy to care for they are tolerant of both over and under watering.,Full Catalog,NaN,"['color', 'hawaiian', 'plant', 'cordyline']",electronics
29,5121322,https://urbantropicals.com/product/excaliber-pink-guava-tree-psidium-guajava,320,Product_urbantropicals.com_September2020.json.gz,excalibur pink guava tree (psidium guajava),"High Yield -Excalibur Pink Guava is a very highly productive tree from Florida, producing medium size fruit for eating fresh off the tree or canning and preserving. The smooth, round shaped fruit is yellowish-green on the exterior with a unique pinkish-cream interior flesh. Fruit typically scales in at 0.8 - 1.3 lbs. and are crispy, tart and aromatic with a sweet juicy pulp. The maturing trunk of this Guava tree is covered in beautiful copper colored bark that peels off regularly to expose shades of pink and green. Light pruning is recommended to develop a strong base in order to support the additional weight during fruiting season.",All Trees,NaN,"['excalibur', 'pink', 'guava', 'tree', 'psidium', 'guajava']",electronics
32,5121322,https://www.estatediamondjewelry.com/product/antique-tiffany-diamond-ring,67,Product_estatediamondjewelry.com_September2020.json.gz,antique tiffany & co. diamond ring,"Antique Tiffany Engagement Ring.Gorgeous Antique Tiffany &amp; Co. old European cut diamond ring. Center diamond is a beautiful old European cut which is 3.83ct, G color, SI1 clarity and is certified by GIA.The three small diamonds that enhance the shank are set in platinum-topped 18K gold and the center diamond is set in platinum prongs. There is so much beauty in the simplicity of this ring and the fire of the diamond. A Stunning Vintage Ring with a magnificent diamond.Circa 1905.To see our complete collection of rare antique tiffany engagement rings, click here.",Estate Diamond Jewelry,NaN,"['antique', 'tiffany', 'co.', 'diamond', 'ring']",electronics
33,5121322,https://www.mornflake.com/product/stoneground-oatmeal-medium-grade-70,9,Product_mornflake.com_September2020.json.gz,stoneground oatmeal medium (grade 70),"Our Medium Oatmeal is the perfect ingredient. As well as delicious porridge, Mornflake Medium Oatmeal is perfect as a coating for fish or meat, ideal for light stuffings and makes a really crunchy crumble topping.",NaN,NaN,"['stoneground', 'oatmeal', 'medium', 'grade', '70']",electronics
...,...,...,...,...,...,...,...,...,...,...
23880,5121322,https://monsterblooms.com/product/sansevieria-ballyi-snake-plant,80,Product_monsterblooms.com_September2020.json.gz,sansevieria ballyi (snake plant),"Ballyi High Color with its cream variegation travels by above ground offsets similar to that of a spider plant. Spring and early summer delicate lightly scented flowers emerge from stalks. Commonly grown as office plants, it is one of those plants that can survive under periods of extreme neglect. Due to its toughness and ease of growth, they are perfect for growing indoors or on the urban balcony.",Full Catalog,NaN,"['sansevieria', 'ballyi', 'snake', 'plant']",random
23881,5121322,https://monsterblooms.com/product/santa-rosa-angel-trumpet-plant-brugmansia,81,Product_monsterblooms.com_September2020.json.gz,santa rosa angel trumpet plant (

In [80]:
# 2200 for around 9622 entries and 299 distinct cluster_ids
df_clusters.drop_duplicates(subset=['table_id'])

,cluster_id,url,row_id,table_id,name,description,brand_x,brand_y,tokens,category
3,16617,https://simplifinetworks.com/v2/point-to-multi-point/266-nanobeam-ac-gen2.html,167,Product_simplifinetworks.com_September2020.json.gz,nanobeam-ac-gen2,NBE-5AC-Gen2The sleek NanoBeam design with proprietary airMAX ac chipset and dedicated management Wi-Fi for easy U Mobile app support and fast setup. This airMAX ac CPE pivots on a ball joint for mounting and alignment flexibility.DATA SHEET,NaN,NaN,['nanobeam-ac-gen2'],electronics
4,323989,https://store.macs-4-u.com.au/usb-c-charge-cable-1-m.html,192,Product_macs-4-u.com.au_September2020.json.gz,usb-c charge cable (1 m),USB-C Charge Cable (1 m),NaN,NaN,"['usb-c', 'charge', 'cable', '1']",electronics
6,345616,https://www.scan.co.uk/products/6tb-wd-red-pro-wd6003ffbx-35-nas-hdd-sata-iii-6gb-s-7200rpm-256mb-cache-shock-sensor-ncq-oem,4945,Product_scan.co.uk_September2020.json.gz,"wd red pro 6tb 3.5\"" sata nas hdd/hard drive","6TB WD Red Pro WD6003FFBX, 3.5\"" NAS HDD, SATA III 6Gb/s, 7200rpm, 256MB Cache, Shock Sensor, NCQ, OEM",WD,NaN,"['wd', 'red', 'pro', '6tb', '3.5\\', ""''"", 'sata', 'hdd/hard', 'drive']",electronics
7,213496,https://de.camerok.com/Sony_SEL30M35_camera_lense,644,Product_camerok.com_September2020.json.gz,sony sel30m35 camera lense,"Dieses Objektiv wurde entwickelt, um bieten vielseitige Hochleistungs-makro-Funktionen in einem kompakten, leichten Körper. Es ist eine echte 1:1 makro-Objektiv mit einer erstaunlichen Naheinstellgrenze von nur knapp 1 Zoll.Die SEL30M35 wurde entwickelt, um bieten vielseitige Hochleistungs-makro-Funktionen in einem Objektiv ist kompakt, leicht und einfach zu bedienen. Es ist eine echte 1:1 makro-Objektiv mit 2,4 Zentimeter minimale Arbeitsabstand ermöglicht winzigen Themen und details zu erbringenden mit hervorragender Auflösung und Kontrast.1:1 AbbildungsmaßstabDie SEL30M35 hat eine 1:1-Abbildungsmaßstab so, dass die Probanden erfasst werden können, werden in voller Größe auf dem Bildsensor der Kamera. Dies macht es einfach, um zu Schießen Qualität makro-Nahaufnahmen von Blumen, Insekten und kleine Objekte.Asphärische/ED-Glas-ElementeDie SEL30M35 bietet eine hervorragende Bildqualität über den gesamten Bildbereich, mit hohem Kontrast und minimale chromatische aberration von unendlich bis 1:1 Vergrößerung bei allen Blendeneinstellungen. ED-Glas verwendet wird, für die G4-element, minimieren die chromatische aberration und die Farbe Blutungen sowie multi-Beschichtung während für besonders saubere, klare rendering.Gemacht für FilmeDie SEL30M35 ebenso führt auch bei Videoaufnahmen. Dank eines internen Schrittmotor und hinten-mit Schwerpunkt design ist optimiert für video-performance für Laufruhe Grundvoraussetzung für hochwertige Film zu erfassen. Darüber hinaus werden die filter Halterung nicht drehen, während der Schwerpunkt für die problemlose polarisierten filter verwenden.Glatte Manuelle FokussierungEin easy-grip-Fokus-ring auf dem Objektivtubus gibt Ihnen eine Feste \""hands-on\"" fühlen sich in der manuellen Fokussierung. Zusätzlich, Direct Manual Focus (DMF) können Sie den Fokus manuell nach AF lock-on, ohne die Notwendigkeit für einen Wechsel. Dies kann sehr nützlich in Situationen, in denen der AF nicht sperren auf das Thema genau, oder wenn AF Schlösser auf einen Punkt, der nicht mit Ihrer Absicht. Elektronische manueller Fokus-Technologie wurde ursprünglich entwickelt für den professionellen Sony Camcorder, um glatte, präzise Fokussierung.",Sony,NaN,"['sony', 'sel30m35', 'camera', 'lense']",electronics
8,167469,https://www.boulanger.com/ref/9000224731?xtor=AL-6875-%5B7%5D-%5B1395090794%5D-%5Bdeeplink%5D&utm_medium=affiliation&utm_source=effinity&utm_campaign=7&utm_content=1395090794,6052,Product_boulanger.com_September2020.json.gz,caméra de sécurité xiaomi mi home security camera 360degrés 1080,Xiaomi -Caméra de sécurité Domestique 360° 1080P- Blanc,NaN,NaN,"['caméra', 'sécurité', 'xiaomi', 'home', 'security', 'camera', '360degrés', '1

In [81]:
df_joined_clothes = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Clothes_clusters_all_8_tables_post_processed_lower_threshold.csv'))
df_joined_clothes['category']='clothes'

In [62]:
df_joined_electronics = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Electronics_clusters_all_8_tables_post_processed_lower_threshold.csv'))
df_joined_electronics['category']='electronics'

In [63]:
df_joined_bikes = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Bikes_clusters_all_8_tables_post_processed_lower_threshold.csv'))
df_joined_bikes['category']='bikes'

In [64]:
df_joined_cars = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Cars_clusters_all_8_tables_post_processed_lower_threshold.csv'))
df_joined_cars['category']='cars'

In [65]:
df_joined_technology = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Technology_clusters_all_8_tables_post_processed_lower_threshold.csv'))
df_joined_technology['category']='technology'

In [66]:
df_joined_drugstore = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Drugstore_clusters_all_8_tables_post_processed_lower_threshold.csv'))
df_joined_drugstore['category']='drugstore'

In [67]:
df_joined_tools = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Tools_clusters_all_8_tables_post_processed_lower_threshold.csv'))
df_joined_tools['category']='tools'

In [68]:
df_joined_random = pd.read_csv(os.path.join(mapping_corpus_path_2, 'Random_clusters_all_8_tables_post_processed_lower_threshold.csv'))
df_joined_random['category']='random'

In [69]:
frames = [df_joined_electronics, df_joined_clothes,df_joined_bikes,df_joined_cars,df_joined_technology,df_joined_drugstore,df_joined_tools,df_joined_random]
df_concat = pd.concat(frames).drop(columns = ['Unnamed: 0','Unnamed: 0.1','Valid'])
df_concat

,cluster_id,url,row_id,table_id,name,description,brand_x,brand_y,tokens,category
0,289820,https://www.tinkco.com/fr/canon-pgi-555pgbk-xxl-cartouche-d-encre-canon-8049b001-noir-pigmentee-37ml-pgi555pgbkxxl,2502,Product_tinkco.com_September2020.json.gz,canon pgi-555pgbk xxl - cartouche d'encrecanon 8049b001 noir pigmentée 37ml,Cartouche originale Canon PGI-555PGBK XXL (PGI555PGBKXXL / 8049B001) de très grande capacité pour imprimante Canon MX725 et MX925. Couleur : noir p...,NaN,NaN,"['canon', 'pgi-555pgbk', 'xxl', 'cartouche', ""d'encrecanon"", '8049b001', 'noir', 'pigmentée', '37ml']",electronics
1,2456244,https://shop-stjohns.simplyforlife.com/quest-chips-cheddar-sour-cream-32g.html,1958,Product_simplyforlife.com_September2020.json.gz,"quest nutrition quest - chips, cheddar & sour cream (32g)",NaN,Quest Nutrition,NaN,"['quest', 'nutrition', 'quest', 'chips', 'cheddar', 'sour', 'cream', '32g']",electronics
2,9885,https://www.tecnoprecios.net/t%C3%B3ner-original/497-toner-canon-723-amarillo-8500-paginas.html,607,Product_tecnoprecios.net_September2020.json.gz,toner canon 723 amarillo 8500 páginas,TONER CANON 723 AMARILLO 8500 PáGINAS LBP7750CDN,NaN,NaN,"['toner', 'canon', '723', 'amarillo', '8500', 'páginas']",electronics
3,16617,https://simplifinetworks.com/v2/point-to-multi-point/266-nanobeam-ac-gen2.html,167,Product_simplifinetworks.com_September2020.json.gz,nanobeam-ac-gen2,NBE-5AC-Gen2The sleek NanoBeam design with proprietary airMAX ac chipset and dedicated management Wi-Fi for easy U Mobile app support and fast setup. This airMAX ac CPE pivots on a ball joint for mounting and alignment flexibility.DATA SHEET,NaN,NaN,['nanobeam-ac-gen2'],electronics
4,323989,https://store.macs-4-u.com.au/usb-c-charge-cable-1-m.html,192,Product_macs-4-u.com.au_September2020.json.gz,usb-c charge cable (1 m),USB-C Charge Cable (1 m),NaN,NaN,"['usb-c', 'charge', 'cable', '1']",electronics
...,...,...,...,...,...,...,...,...,...,...
23904,36306411,https://www.mainstreetfloorcoveringvermont.com/american-olean-mirasol-whites--creams-silver-marble-12-tile-flooring-ml7212121p,35,Product_mainstreetfloorcoveringvermont.com_September2020.json.gz,mirasol silver marble ml72,"Main Street Floor Covering in Essex Junction has a top selection of American Olean Ceramic & Porcelain Tile Flooring, including Mirasol™ Silver Marble in 12\"" x 12\""",American Olean,american olean,"['mirasol', 'silver', 'marble', 'ml72']",random
23905,62520180,https://www.thefloordrs.com/american-olean-quarry-naturals-beiges--browns-desert--2-tile-flooring-0n0322chipa,59,Product_thefloordrs.com_September2020.json.gz,quarry naturals desertn03 1,"The Floor Doctors in Des Moines has a top selection of American Olean Ceramic & Porcelain Tile Flooring, including Quarry Naturals® Desert * in 2\"" x 2\""",American Olean,american olean,"['quarry', 'naturals', 'desertn03', '1']",random
23906,56187443,https://www.amitybicycles.com/product/garmin-edge-510-silicone-case-198697-1.htm,353,Product_amitybicycles.com_September2020.json.gz,edge 510 silicone case,"Protect your Garmin Edge 510 with a form-fitting, silicone case and keep your mind at ease. Choose from a variety of cool colors to match your personal style.",NaN,NaN,"['edge', '510', 'silicone', 'case']",random
23907,2778331,https://www.curiousemporium.co.uk/products/lego-75957-harry-potter-the-knight-bus,59,Product_curiousemporium.co.uk_September2020.json.gz,lego 75957 harry potter the knight bus,"Give your young witch or wizard the ride of their life aboard the LEGO® Harry Potter™ 75957 Knight Bus™! When kids catch the purple triple-decker bus, they know they're in for a wild ride. With a hinged side panel for easy access to the interior, and a removable roof to reveal the upper deck, this magical toy bus playset is bursting with play potential. Once Harry's trunk has been loaded, it's time to hold tight. As the vehicle races away, swerving around corners and bouncing over bumps, kids will love how the bed slides back and forth and the

In [70]:
df_final_lower = pd.read_excel("Final_lower threshold.xlsx")

In [71]:
df_clusters = df_concat[df_concat['cluster_id'].isin(df_final_lower['cluster_id'].to_list())]

In [72]:
df_clusters

,cluster_id,url,row_id,table_id,name,description,brand_x,brand_y,tokens,category
4,323989,https://store.macs-4-u.com.au/usb-c-charge-cable-1-m.html,192,Product_macs-4-u.com.au_September2020.json.gz,usb-c charge cable (1 m),USB-C Charge Cable (1 m),NaN,NaN,"['usb-c', 'charge', 'cable', '1']",electronics
6,345616,https://www.scan.co.uk/products/6tb-wd-red-pro-wd6003ffbx-35-nas-hdd-sata-iii-6gb-s-7200rpm-256mb-cache-shock-sensor-ncq-oem,4945,Product_scan.co.uk_September2020.json.gz,"wd red pro 6tb 3.5\"" sata nas hdd/hard drive","6TB WD Red Pro WD6003FFBX, 3.5\"" NAS HDD, SATA III 6Gb/s, 7200rpm, 256MB Cache, Shock Sensor, NCQ, OEM",WD,NaN,"['wd', 'red', 'pro', '6tb', '3.5\\', ""''"", 'sata', 'hdd/hard', 'drive']",electronics
7,213496,https://de.camerok.com/Sony_SEL30M35_camera_lense,644,Product_camerok.com_September2020.json.gz,sony sel30m35 camera lense,"Dieses Objektiv wurde entwickelt, um bieten vielseitige Hochleistungs-makro-Funktionen in einem kompakten, leichten Körper. Es ist eine echte 1:1 makro-Objektiv mit einer erstaunlichen Naheinstellgrenze von nur knapp 1 Zoll.Die SEL30M35 wurde entwickelt, um bieten vielseitige Hochleistungs-makro-Funktionen in einem Objektiv ist kompakt, leicht und einfach zu bedienen. Es ist eine echte 1:1 makro-Objektiv mit 2,4 Zentimeter minimale Arbeitsabstand ermöglicht winzigen Themen und details zu erbringenden mit hervorragender Auflösung und Kontrast.1:1 AbbildungsmaßstabDie SEL30M35 hat eine 1:1-Abbildungsmaßstab so, dass die Probanden erfasst werden können, werden in voller Größe auf dem Bildsensor der Kamera. Dies macht es einfach, um zu Schießen Qualität makro-Nahaufnahmen von Blumen, Insekten und kleine Objekte.Asphärische/ED-Glas-ElementeDie SEL30M35 bietet eine hervorragende Bildqualität über den gesamten Bildbereich, mit hohem Kontrast und minimale chromatische aberration von unendlich bis 1:1 Vergrößerung bei allen Blendeneinstellungen. ED-Glas verwendet wird, für die G4-element, minimieren die chromatische aberration und die Farbe Blutungen sowie multi-Beschichtung während für besonders saubere, klare rendering.Gemacht für FilmeDie SEL30M35 ebenso führt auch bei Videoaufnahmen. Dank eines internen Schrittmotor und hinten-mit Schwerpunkt design ist optimiert für video-performance für Laufruhe Grundvoraussetzung für hochwertige Film zu erfassen. Darüber hinaus werden die filter Halterung nicht drehen, während der Schwerpunkt für die problemlose polarisierten filter verwenden.Glatte Manuelle FokussierungEin easy-grip-Fokus-ring auf dem Objektivtubus gibt Ihnen eine Feste \""hands-on\"" fühlen sich in der manuellen Fokussierung. Zusätzlich, Direct Manual Focus (DMF) können Sie den Fokus manuell nach AF lock-on, ohne die Notwendigkeit für einen Wechsel. Dies kann sehr nützlich in Situationen, in denen der AF nicht sperren auf das Thema genau, oder wenn AF Schlösser auf einen Punkt, der nicht mit Ihrer Absicht. Elektronische manueller Fokus-Technologie wurde ursprünglich entwickelt für den professionellen Sony Camcorder, um glatte, präzise Fokussierung.",Sony,NaN,"['sony', 'sel30m35', 'camera', 'lense']",electronics
8,167469,https://www.boulanger.com/ref/9000224731?xtor=AL-6875-%5B7%5D-%5B1395090794%5D-%5Bdeeplink%5D&utm_medium=affiliation&utm_source=effinity&utm_campaign=7&utm_content=1395090794,6052,Product_boulanger.com_September2020.json.gz,caméra de sécurité xiaomi mi home security camera 360degrés 1080,Xiaomi -Caméra de sécurité Domestique 360° 1080P- Blanc,NaN,NaN,"['caméra', 'sécurité', 'xiaomi', 'home', 'security', 'camera', '360degrés', '1080']",electronics
9,2374515,https://www.tecnoprecios.net/tinta-original/860-multipack-canon-pg-545-cl-546-blister-mg2450-mg2550.html,611,Product_tecnoprecios.net_September2020.json.gz,cartucho tinta canon pg545 negro + cl546 color,MULTIPACK CANON PG-545/CL-546 BLISTER MG2450-MG2550,NaN,NaN,"['cartucho', 'tinta', 'canon', 'pg545', 'negro', 'cl546', 'color']",electronics
...,...,...,...,...,...,...,...,...,...,...
23902,617074,https://www.

In [73]:
df_clusters[df_clusters['cluster_id']==113942]

,cluster_id,url,row_id,table_id,name,description,brand_x,brand_y,tokens,category


In [74]:
# 2200 for around 9622 entries and 299 distinct cluster_ids
df_clusters.drop_duplicates(subset=['table_id'])

,cluster_id,url,row_id,table_id,name,description,brand_x,brand_y,tokens,category
4,323989,https://store.macs-4-u.com.au/usb-c-charge-cable-1-m.html,192,Product_macs-4-u.com.au_September2020.json.gz,usb-c charge cable (1 m),USB-C Charge Cable (1 m),NaN,NaN,"['usb-c', 'charge', 'cable', '1']",electronics
6,345616,https://www.scan.co.uk/products/6tb-wd-red-pro-wd6003ffbx-35-nas-hdd-sata-iii-6gb-s-7200rpm-256mb-cache-shock-sensor-ncq-oem,4945,Product_scan.co.uk_September2020.json.gz,"wd red pro 6tb 3.5\"" sata nas hdd/hard drive","6TB WD Red Pro WD6003FFBX, 3.5\"" NAS HDD, SATA III 6Gb/s, 7200rpm, 256MB Cache, Shock Sensor, NCQ, OEM",WD,NaN,"['wd', 'red', 'pro', '6tb', '3.5\\', ""''"", 'sata', 'hdd/hard', 'drive']",electronics
7,213496,https://de.camerok.com/Sony_SEL30M35_camera_lense,644,Product_camerok.com_September2020.json.gz,sony sel30m35 camera lense,"Dieses Objektiv wurde entwickelt, um bieten vielseitige Hochleistungs-makro-Funktionen in einem kompakten, leichten Körper. Es ist eine echte 1:1 makro-Objektiv mit einer erstaunlichen Naheinstellgrenze von nur knapp 1 Zoll.Die SEL30M35 wurde entwickelt, um bieten vielseitige Hochleistungs-makro-Funktionen in einem Objektiv ist kompakt, leicht und einfach zu bedienen. Es ist eine echte 1:1 makro-Objektiv mit 2,4 Zentimeter minimale Arbeitsabstand ermöglicht winzigen Themen und details zu erbringenden mit hervorragender Auflösung und Kontrast.1:1 AbbildungsmaßstabDie SEL30M35 hat eine 1:1-Abbildungsmaßstab so, dass die Probanden erfasst werden können, werden in voller Größe auf dem Bildsensor der Kamera. Dies macht es einfach, um zu Schießen Qualität makro-Nahaufnahmen von Blumen, Insekten und kleine Objekte.Asphärische/ED-Glas-ElementeDie SEL30M35 bietet eine hervorragende Bildqualität über den gesamten Bildbereich, mit hohem Kontrast und minimale chromatische aberration von unendlich bis 1:1 Vergrößerung bei allen Blendeneinstellungen. ED-Glas verwendet wird, für die G4-element, minimieren die chromatische aberration und die Farbe Blutungen sowie multi-Beschichtung während für besonders saubere, klare rendering.Gemacht für FilmeDie SEL30M35 ebenso führt auch bei Videoaufnahmen. Dank eines internen Schrittmotor und hinten-mit Schwerpunkt design ist optimiert für video-performance für Laufruhe Grundvoraussetzung für hochwertige Film zu erfassen. Darüber hinaus werden die filter Halterung nicht drehen, während der Schwerpunkt für die problemlose polarisierten filter verwenden.Glatte Manuelle FokussierungEin easy-grip-Fokus-ring auf dem Objektivtubus gibt Ihnen eine Feste \""hands-on\"" fühlen sich in der manuellen Fokussierung. Zusätzlich, Direct Manual Focus (DMF) können Sie den Fokus manuell nach AF lock-on, ohne die Notwendigkeit für einen Wechsel. Dies kann sehr nützlich in Situationen, in denen der AF nicht sperren auf das Thema genau, oder wenn AF Schlösser auf einen Punkt, der nicht mit Ihrer Absicht. Elektronische manueller Fokus-Technologie wurde ursprünglich entwickelt für den professionellen Sony Camcorder, um glatte, präzise Fokussierung.",Sony,NaN,"['sony', 'sel30m35', 'camera', 'lense']",electronics
8,167469,https://www.boulanger.com/ref/9000224731?xtor=AL-6875-%5B7%5D-%5B1395090794%5D-%5Bdeeplink%5D&utm_medium=affiliation&utm_source=effinity&utm_campaign=7&utm_content=1395090794,6052,Product_boulanger.com_September2020.json.gz,caméra de sécurité xiaomi mi home security camera 360degrés 1080,Xiaomi -Caméra de sécurité Domestique 360° 1080P- Blanc,NaN,NaN,"['caméra', 'sécurité', 'xiaomi', 'home', 'security', 'camera', '360degrés', '1080']",electronics
9,2374515,https://www.tecnoprecios.net/tinta-original/860-multipack-canon-pg-545-cl-546-blister-mg2450-mg2550.html,611,Product_tecnoprecios.net_September2020.json.gz,cartucho tinta canon pg545 negro + cl546 color,MULTIPACK CANON PG-545/CL-546 BLISTER MG2450-MG2550,NaN,NaN,"['cartucho', 'tinta', 'canon', 'pg545', 'negro', 'cl546', 'color']",electronics
...,...,...,...,...,...,...,...,...,...,...
23759,2795175,https://www

In [75]:
df_joined_cars

,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,table_id,Valid,name,description,brand_x,brand_y,tokens,category
0,2,49017,5121322,https://www.akcomicshop.com/7-wonders.html,862,Product_akcomicshop.com_September2020.json.gz,1,asmodee 7 wonders,Base Game3-7 playersAges 13+30 mins,Asmodee,NaN,"['asmodee', '7', 'wonders']",cars
1,6,53396,5121322,https://monsterblooms.com/product/tri-color-ti-hawaiian-ti-plant-cordyline,198,Product_monsterblooms.com_September2020.json.gz,1,tri color ti hawaiian ti plant (cordyline),Tricolor Foliage. Cordyline Tricolor is a mid sized easy to grow Cordyline. Leaves sport pink cream green and maroon striated coloration which darken slightly with age. Mature height is around 4 feet. Hawaiian Ti plants are the perfect plant choice to add lush tropical colors to any indoor or outdoor environment. Easy to care for they are tolerant of both over and under watering.,Full Catalog,NaN,"['color', 'hawaiian', 'plant', 'cordyline']",cars
2,9,54388,2158913,https://www.fangirlcollectables.com/product-page/stormtrooper-head-necklace,31,Product_fangirlcollectables.com_September2020.json.gz,1,stormtrooper head necklace,Cartoon Star Wars Metal Charms Classic Pendants Jewelry,NaN,NaN,"['stormtrooper', 'head', 'necklace']",cars
3,14,57801,5121322,https://www.mornflake.com/product/stoneground-oatmeal-medium-grade-70,9,Product_mornflake.com_September2020.json.gz,1,stoneground oatmeal medium (grade 70),"Our Medium Oatmeal is the perfect ingredient. As well as delicious porridge, Mornflake Medium Oatmeal is perfect as a coating for fish or meat, ideal for light stuffings and makes a really crunchy crumble topping.",NaN,NaN,"['stoneground', 'oatmeal', 'medium', 'grade', '70']",cars
4,16,61234,2158913,https://www.timeboulevard.com/product-page/jaeger-lecoultre-memovox-wrist-alarm,9,Product_timeboulevard.com_September2020.json.gz,1,jaeger lecoultre memovox,Jaeger-LeCoultre Memovox is one of the most iconic wrist alarm in horological history. 35mm 10k gold filled case is etched 'Cased &amp; Timed in USA'. It houses the legendary manual-wind calibre 814 movement.,NaN,NaN,"['jaeger', 'lecoultre', 'memovox']",cars
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4612,15252,24671086,5121322,https://spectrumtelescope.com/product/glass-solar-filter-st1150g,30,Product_spectrumtelescope.com_September2020.json.gz,1,glass solar filter st1150g,"Your O.D. Measurement:11 1/8” (283mm)to11 3/8” (289mm) Fits:Celestron 10\"" Newt;Orion SkyWatcher 10\""",Spectrum Telescope,NaN,"['glass', 'solar', 'filter', 'st1150g']",cars
4613,15254,24672021,3604187,https://www.jaguarlv.com/inventory/used-2018-mercedes-benz-gle-gle-350-4matic%C2%AE-4d-sport-utility-4jgda5hb7jb171041,4,Product_jaguarlv.com_September2020.json.gz,1,pre-owned 2018 mercedes-benz gle gle 350 with navigation,Clean CARFAX. White 2018 Mercedes-Benz GLE GLE 350 4MATIC® 4MATIC® 7G-TRONIC 7-Speed Automatic 3.5L V6 DOHC 24V,Mercedes-Benz,mercedes-benz,"['pre-owned', '2018', 'mercedes-benz', 'gle', 'gle', '350', 'navigation']",cars
4614,15256,24672714,6058910,https://www.sullivanparkhillchevy.com/inventory/used-2019-hyundai-tucson-wagon-4-dr-km8j23a46ku894451,22,Product_sullivanparkhillchevy.com_September2020.json.gz,1,pre-owned 2019 hyundai tucson wagon 4 dr.,"2019 Hyundai Tucson SE Clean CarFax Report, Alloy wheels, Brake assist, Electronic Stability Control, Exterior Parking Camera Rear, Fully automatic headlights, Option Group 01, Overhead airbag, Speed control, Speed-sensing steering, Steering wheel mounted audio controls, Traction control.CARFAX One-Owner. Clean CARFAX.Sullivan-Parkhill Precision Pricing: At Sullivan-Parkhill, we want you to know that all our vehicles are priced at a competitive value position to the market. We use an independent 3rd party software to research internet listings on all vehicles in the market so we can ensure that our prices are the most competitive out there. We do this simply so people choose us when they start searching for their next car.Locall

In [11]:
table_counts = df_electronics_clusters.groupby('table_id').count()

In [12]:
table_counts[table_counts ['cluster_id'].sort_values(ascending=False) > 15]

<ipython-input-12-a30fe86562f9>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  table_counts[table_counts ['cluster_id'].sort_values(ascending=False) > 15]


,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,Valid,name,description,brand,tokens
table_id,,,,,,,,,,
Product_allstars.com.sg_September2020.json.gz,19,19,19,19,19,19,19,19,0,19
Product_bellaphoto.com.br_September2020.json.gz,16,16,16,16,16,16,16,16,0,16
Product_bensonappliance.com_September2020.json.gz,20,20,20,20,20,20,20,0,0,20
Product_bilisimport.com.tr_September2020.json.gz,17,17,17,17,17,17,17,17,0,17
Product_bt.com_September2020.json.gz,26,26,26,26,26,26,26,26,0,26
Product_camerashopmuskegon.com_September2020.json.gz,16,16,16,16,16,16,16,16,0,16
Product_cclonline.com_September2020.json.gz,26,26,26,26,26,26,26,26,0,26
Product_crutchfield.com_September2020.json.gz,23,23,23,23,23,23,23,23,0,23
Product_diamondscamera.com.au_September2020.json.gz,30,30,30,30,30,30,30,30,30,30


In [13]:
df_electronics_clusters[['cluster_id','row_id','table_id','name']]

,cluster_id,row_id,table_id,name
0,1001446,90,Product_bts.com.au_September2020.json.gz,apple lightning digital av adapter
1,2859551,511,Product_icentre.com.mt_September2020.json.gz,apple lightning to usb cable
2,323989,192,Product_macs-4-u.com.au_September2020.json.gz,usb-c charge cable (1 m)
6,2374515,611,Product_tecnoprecios.net_September2020.json.gz,cartucho tinta canon pg545 negro + cl546 color
9,144625,764,Product_cameraworld.co.uk_September2020.json.gz,nikon 28-300mm f3.5-5.6g ed vr af-s nikkor lens
...,...,...,...,...
9312,3756053,110,Product_brandnameappliance.net_September2020.json.gz,23 cu. ft. counter depth 4-door french door refrigerator in stainless steel
9315,1929047,57,Product_kartmy.com_September2020.json.gz,samsung 970 evo 500gb nvme m.2 high speed solid state drive with v-nand
9318,32592,230,Product_copaco.com_September2020.json.gz,"western digital wd red pro 3.5"" 12000 gb serial ata iii"
9321,48051973,24,Product_ktrworld.com_September2020.json.gz,samsung gear vr virtual reality headset


In [42]:
df_cluster_323989 = df_electronics_clusters[df_electronics_clusters['cluster_id']==323989][['cluster_id','row_id','table_id','name']]

In [28]:
def jaccard_similarity_score(original, translation):
    intersect = set(original).intersection(set(translation))
    union = set(original).union(set(translation))
    try:
        return len(intersect) / len(union)
    except ZeroDivisionError:
        return 0

In [29]:
def remove_punctuation(token_vector):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in string.punctuation])

In [30]:
def remove_stopwords(token_vector, stopwords_list):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in stopwords_list])

In [47]:
df_cluster_323989['product_tokens'] = df_cluster_323989['name'].apply(lambda row: row.lower()).apply(lambda row: word_tokenize(row))
df_cluster_323989['product_tokens'] = remove_stopwords(df_cluster_323989['product_tokens'],stopwords.words())
df_cluster_323989['product_tokens'] = remove_punctuation (df_cluster_323989['product_tokens'])
df_cluster_323989.reset_index(inplace=True)

In [48]:
df_cluster_323989

,index,cluster_id,row_id,table_id,name,product_tokens
0,18006,323989,192,Product_macs-4-u.com.au_September2020.json.gz,USB-C Charge Cable (1 m),"[usb-c, charge, cable, 1]"
1,1318608,323989,1064,Product_retravision.com.au_September2020.json.gz,Apple USB-C Charge Cable (1M),"[apple, usb-c, charge, cable, 1m]"
2,2826897,323989,152,Product_pskmegastore.com_September2020.json.gz,Apple MUF72ZM/A cavo USB 1 m USB C Bianco,"[apple, muf72zm/a, cavo, usb, 1, usb, bianco]"
3,3207585,323989,9,Product_ilovecomputers.com.au_September2020.json.gz,Apple Apple USB-C Charge Cable (1 m),"[apple, apple, usb-c, charge, cable, 1]"
4,12580514,323989,146,Product_mcarerental.com_September2020.json.gz,Apple USB-C Charge Cable 1m,"[apple, usb-c, charge, cable, 1m]"
5,15121729,323989,321,Product_alza.co.uk_September2020.json.gz,Apple USB-C Charging Cable 1m,"[apple, usb-c, charging, cable, 1m]"
6,15290618,323989,20,Product_lecbroome.com.au_September2020.json.gz,USB-C Charge Cable (2M),"[usb-c, charge, cable, 2m]"
7,16279424,323989,8,Product_expotronica.com_September2020.json.gz,Apple cable conexión usb tipo c a usb-c 1m color blanco,"[apple, cable, conexión, usb, tipo, usb-c, 1m, color, blanco]"
8,17518365,323989,308,Product_gms-store.com_September2020.json.gz,Cabo USB-C para USB-C 1 metro,"[cabo, usb-c, usb-c, 1, metro]"
9,17912158,323989,5743,Product_euro.com.pl_September2020.json.gz,Apple Kabel USB-C,"[apple, kabel, usb-c]"


In [53]:
df_cluster_323989['jaccard_score'] = df_cluster_323989['product_tokens'].apply(lambda row: jaccard_similarity_score(row,df_cluster_323989.iloc[0]['product_tokens']) )

In [54]:
df_cluster_323989

,index,cluster_id,row_id,table_id,name,product_tokens,jaccard_score
0,18006,323989,192,Product_macs-4-u.com.au_September2020.json.gz,USB-C Charge Cable (1 m),"[usb-c, charge, cable, 1]",1.000000
1,1318608,323989,1064,Product_retravision.com.au_September2020.json.gz,Apple USB-C Charge Cable (1M),"[apple, usb-c, charge, cable, 1m]",0.500000
2,2826897,323989,152,Product_pskmegastore.com_September2020.json.gz,Apple MUF72ZM/A cavo USB 1 m USB C Bianco,"[apple, muf72zm/a, cavo, usb, 1, usb, bianco]",0.111111
3,3207585,323989,9,Product_ilovecomputers.com.au_September2020.json.gz,Apple Apple USB-C Charge Cable (1 m),"[apple, apple, usb-c, charge, cable, 1]",0.800000
4,12580514,323989,146,Product_mcarerental.com_September2020.json.gz,Apple USB-C Charge Cable 1m,"[apple, usb-c, charge, cable, 1m]",0.500000
5,15121729,323989,321,Product_alza.co.uk_September2020.json.gz,Apple USB-C Charging Cable 1m,"[apple, usb-c, charging, cable, 1m]",0.285714
6,15290618,323989,20,Product_lecbroome.com.au_September2020.json.gz,USB-C Charge Cable (2M),"[usb-c, charge, cable, 2m]",0.600000
7,16279424,323989,8,Product_expotronica.com_September2020.json.gz,Apple cable conexión usb tipo c a usb-c 1m color blanco,"[apple, cable, conexión, usb, tipo, usb-c, 1m, color, blanco]",0.181818
8,17518365,323989,308,Product_gms-store.com_September2020.json.gz,Cabo USB-C para USB-C 1 metro,"[cabo, usb-c, usb-c, 1, metro]",0.333333
9,17912158,323989,5743,Product_euro.com.pl_September2020.json.gz,Apple Kabel USB-C,"[apple, kabel, usb-c]",0.166667


In [22]:
df_joined_clothes = pd.read_csv(os.path.join(mapping_corpus_path_2, 'clothes_clusters_all_8_tables_post_processed.csv'))

In [23]:
df_clothes_final = pd.read_excel("Final_Clothes_v4.xlsx")

In [24]:
df_clothes_clusters = df_joined_clothes[df_joined_clothes['cluster_id'].isin(df_clothes_final['cluster_id'].to_list())]

In [26]:
df_clothes_clusters

,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,table_id,Valid,name,description,brand_x,brand_y,tokens
0,4,15925,78110534,https://www.cn.forzieri.com/chn/product_view.asp?l=chn&c=chn&dept_id=18&sku=dg130320-027-00,3400,Product_forzieri.com_September2020.json.gz,1,"small \""devotion\"" bag","Leather handle, detachable shoulder strap, front plate with logo jewel heart, magnetic closure, inner flat pocket, leather inner, gold finish metal accessories, made in italy.Sensorial gift wrap available",Dolce & Gabbana żΰ,NaN,"['small', ""''"", 'devotion\\', ""''"", 'bag']"
15,28,73795,58592784,https://sneaksa.com/product/nike-air-force-1-mid,45,Product_sneaksa.com_September2020.json.gz,1,nike air force 1 mid,"Le basketball dans le parc, le barbecue du dimanche et le soleil. Le charme continue à opérer avec la Nike Air Force 1 ’07. Cette silhouette emblématique du basketball apporte un nouveau souffle à ses éléments les plus remarquables : un cuir impeccable et un coloris blanc intégral pour une allure exceptionnelle sur le terrain comme au quotidien.",NaN,NaN,"['nike', 'air', 'force', '1', 'mid']"
16,30,76085,58083280,https://gamarraecommerce.com/producto/air-force-1-low-white-07,46,Product_gamarraecommerce.com_September2020.json.gz,1,air force 1 low white '07,"CODE 315122-111 COLORWAY WHITE/WHITE RETAIL PRICE $100These Products are supplied by a premium shoe marketplace, which offers the most coveted and hard-to-find footwear in the world.SIZE GUIDE: US (MEN) / US (WOMEN) / EUR",NaN,NaN,"['air', 'force', '1', 'low', 'white', ""'07""]"
22,40,96870,22653830,https://www.fitnewsole.com/travis-scott-x-nike-air-force-1-low-cactus-jack-cn2405-900-new-sale,121,Product_fitnewsole.com_September2020.json.gz,1,"travis scott x nike air force 1 low \""cactus jack\"" cn2405-900 new sale","A light brown canvas heel is covered in various Cactus Jack Records branding. Other details include a light brown canvas Swoosh, wool heel tab, plaid tongue, pink side panels, and dark grey leather toe box. A buckle with brass hardware has also been added to the heel.",NaN,NaN,"['travis', 'scott', 'x', 'nike', 'air', 'force', '1', 'low', ""''"", 'cactus', 'jack\\', ""''"", 'cn2405-900', 'new']"
32,57,139940,66437123,https://www.solehello.com/product/nike-air-mx-97-worldwide-white-blue-fury-black-volt-for-sale,429,Product_solehello.com_September2020.json.gz,1,"nike air max 97 “worldwide\"" white/blue fury-black-volt for sale","Dressed in a White, Blue Fury, Black, and Volt color scheme. This edition of the Nike Air Max 97 features a White upper paired with Blue and Volt contrasting accents with a Crimson tag on the lateral and “Worldwide” text on the Black heel tab.",NaN,nike,"['nike', 'air', 'max', '97', '“', 'worldwide\\', ""''"", 'white/blue', 'fury-black-volt']"
...,...,...,...,...,...,...,...,...,...,...,...,...
6008,9704,24624991,70109,https://da.ulapop.com/products/31872_dkblk,74,Product_ulapop.com_September2020.json.gz,1,coach - woman clutch bag,Køn: Kvindetype: Kobling Materiale: læder Vigtigste fastgørelse: metallisk Skulderrem: aftagelig skulderrem Indvendig: organiseret indvendigt Indvendige lommer: 2 Udvendige lommer: 1 Bredde cm: 18 Højde cm: 10 Dybde cm: 4 Detaljer: applikationer Støvtaske inkluderet synligt logo,NaN,NaN,"['coach', 'woman', 'clutch', 'bag']"
6011,9707,24627376,1784229,https://www.thesoleline.com/online/nike-air-vapormax-2020-summit-white-for-sale,109,Product_thesoleline.com_September2020.json.gz,1,nike air vapormax 2020 “summit white” for sale,This offering of theVaporMax 2020features a Summit White with Pink and Blue knits covering its Flyknit upper constructed of recycled polyester yarns with FlyEase lacing tech. A marbleized heel atop its signature recycled full-length VaporMax Air unit completes the design.,NaN,nike,"['nike', 'air', 'vapormax', '2020', '“', 'summit', 'white', '”']"
6014,9710,24628836,1916084,https://www.anytimewatches.com/en/tissot-t063-610-11-038-00.html,40,Product_anytimewatches.com_September2020.json.gz,1,tissot 

In [27]:
# 2200 for around 9622 entries and 299 distinct cluster_ids
df_clothes_clusters.drop_duplicates(subset=['table_id'])

,Unnamed: 0,Unnamed: 0.1,cluster_id,url,row_id,table_id,Valid,name,description,brand_x,brand_y,tokens
0,4,15925,78110534,https://www.cn.forzieri.com/chn/product_view.asp?l=chn&c=chn&dept_id=18&sku=dg130320-027-00,3400,Product_forzieri.com_September2020.json.gz,1,"small \""devotion\"" bag","Leather handle, detachable shoulder strap, front plate with logo jewel heart, magnetic closure, inner flat pocket, leather inner, gold finish metal accessories, made in italy.Sensorial gift wrap available",Dolce & Gabbana żΰ,NaN,"['small', ""''"", 'devotion\\', ""''"", 'bag']"
15,28,73795,58592784,https://sneaksa.com/product/nike-air-force-1-mid,45,Product_sneaksa.com_September2020.json.gz,1,nike air force 1 mid,"Le basketball dans le parc, le barbecue du dimanche et le soleil. Le charme continue à opérer avec la Nike Air Force 1 ’07. Cette silhouette emblématique du basketball apporte un nouveau souffle à ses éléments les plus remarquables : un cuir impeccable et un coloris blanc intégral pour une allure exceptionnelle sur le terrain comme au quotidien.",NaN,NaN,"['nike', 'air', 'force', '1', 'mid']"
16,30,76085,58083280,https://gamarraecommerce.com/producto/air-force-1-low-white-07,46,Product_gamarraecommerce.com_September2020.json.gz,1,air force 1 low white '07,"CODE 315122-111 COLORWAY WHITE/WHITE RETAIL PRICE $100These Products are supplied by a premium shoe marketplace, which offers the most coveted and hard-to-find footwear in the world.SIZE GUIDE: US (MEN) / US (WOMEN) / EUR",NaN,NaN,"['air', 'force', '1', 'low', 'white', ""'07""]"
22,40,96870,22653830,https://www.fitnewsole.com/travis-scott-x-nike-air-force-1-low-cactus-jack-cn2405-900-new-sale,121,Product_fitnewsole.com_September2020.json.gz,1,"travis scott x nike air force 1 low \""cactus jack\"" cn2405-900 new sale","A light brown canvas heel is covered in various Cactus Jack Records branding. Other details include a light brown canvas Swoosh, wool heel tab, plaid tongue, pink side panels, and dark grey leather toe box. A buckle with brass hardware has also been added to the heel.",NaN,NaN,"['travis', 'scott', 'x', 'nike', 'air', 'force', '1', 'low', ""''"", 'cactus', 'jack\\', ""''"", 'cn2405-900', 'new']"
32,57,139940,66437123,https://www.solehello.com/product/nike-air-mx-97-worldwide-white-blue-fury-black-volt-for-sale,429,Product_solehello.com_September2020.json.gz,1,"nike air max 97 “worldwide\"" white/blue fury-black-volt for sale","Dressed in a White, Blue Fury, Black, and Volt color scheme. This edition of the Nike Air Max 97 features a White upper paired with Blue and Volt contrasting accents with a Crimson tag on the lateral and “Worldwide” text on the Black heel tab.",NaN,nike,"['nike', 'air', 'max', '97', '“', 'worldwide\\', ""''"", 'white/blue', 'fury-black-volt']"
...,...,...,...,...,...,...,...,...,...,...,...,...
5926,9566,24272328,944406,https://www.thefragrancecounter.co.uk/dolce-and-gabbana-pour-homme-edt-75ml.html,314,Product_thefragrancecounter.co.uk_September2020.json.gz,1,dolce and gabbana pour homme edt 75ml,Buy cheap aftershave from Dolce and Gabbana on The Fragrance Counter. FREE Delivery. Many items with FREE gift with purchase. Rated 5* on Trustpilot.,Dolce and Gabbana,NaN,"['dolce', 'gabbana', 'homme', 'edt', '75ml']"
5934,9577,24313111,188731,https://www.vminnovations.com/Product_179028/Oakley-Standard-Fit-Holbrook-Sunglasses-Matte-Black-Prizm-Ruby-Open-Box-.html,937,Product_vminnovations.com_September2020.json.gz,1,"oakley standard fit holbrook sunglasses, matte black/prizm ruby (open box)","Item is open box, unharmed and in very good condition. All accessories or parts are included with the item.Classic design inspired by movie stars of the '40s, '50s, and '60s100% UV protection coating shields your eyes from the sun; great for people with lighter eye colors/light sensitivityFrame Color: Matte blackLens color: Prizm rubyFit: StandardLight Transmission: 17%Light Conditions: Bright lightContrast: IncreasedProduct Condition: Open BoxLike newProduct Wa

In [ ]:
df_clothes_clusters.drop_duplicates(subset=['table_id'])['table_id']

In [ ]:
table_counts = df_clothes_clusters.groupby('table_id').count()

In [ ]:
table_counts[table_counts ['cluster_id'].sort_values(ascending=False) > 15]

In [28]:
df_clothes_clusters[['cluster_id','row_id','table_id','name']]

,cluster_id,row_id,table_id,name
0,78110534,3400,Product_forzieri.com_September2020.json.gz,"small \""devotion\"" bag"
15,58592784,45,Product_sneaksa.com_September2020.json.gz,nike air force 1 mid
16,58083280,46,Product_gamarraecommerce.com_September2020.json.gz,air force 1 low white '07
22,22653830,121,Product_fitnewsole.com_September2020.json.gz,"travis scott x nike air force 1 low \""cactus jack\"" cn2405-900 new sale"
32,66437123,429,Product_solehello.com_September2020.json.gz,"nike air max 97 “worldwide\"" white/blue fury-black-volt for sale"
...,...,...,...,...
6008,70109,74,Product_ulapop.com_September2020.json.gz,coach - woman clutch bag
6011,1784229,109,Product_thesoleline.com_September2020.json.gz,nike air vapormax 2020 “summit white” for sale
6014,1916084,40,Product_anytimewatches.com_September2020.json.gz,tissot tradition t063.610.11.038.00
6015,1315657,123,Product_aufreshscents.com_September2020.json.gz,prada luna rossa sport by prada edt spray 3.4 oz
